Initial Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone git repo
!git clone https://github.com/ethanrasmussen/llm_radiology.git

# Enter correct directory
%cd llm_radiology/
%ls

Mounted at /content/drive
Cloning into 'llm_radiology'...
remote: Enumerating objects: 591, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 591 (delta 19), reused 19 (delta 7), pack-reused 556 (from 1)
Receiving objects: 100% (591/591), 760.74 KiB | 24.54 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/content/llm_radiology
CXRMetric/             finetune.sh*                  report_cleaning.py
dl4h/                  format_llama_input.py         requirements.txt
eval_requirements.txt  image_model/                  utils_finetune.py
evaluate.py            pragmatic_llama_inference.py  utils.py
finetune.py            prompts/
finetune_qLoRA.py      README.md


In [ ]:
# Install project requirements
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.2 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement tifffil

Report Cleaning & Data Formatting for Llama Finetune

In [ ]:
!mkdir /content/drive/MyDrive/LLM_CXR/cleaned_reports/

In [ ]:
!pip install -q datasets deepspeed

In [ ]:
!deepspeed --num_gpus=1 report_cleaning.py --chexbert_path /content/drive/MyDrive/LLM_CXR/chexbert.pth --per_device_eval_batch_size 1 \
  --dataset_path /content/drive/MyDrive/LLM_CXR/PARSED_DATASET/COMPILED_GROUND_TRUTH_REPORTS.csv --output_dir /content/drive/MyDrive/LLM_CXR/cleaned_reports/

In [ ]:
!python format_llama_input.py --indication_path /content/drive/MyDrive/LLM_CXR/cleaned_reports/manually_prepared/FINETUNE_INDICATIONS.csv --impression_path /content/drive/MyDrive/LLM_CXR/cleaned_reports/manually_prepared/FINETUNE_IMPRESSIONS.csv --outpath /content/drive/MyDrive/LLM_CXR/cleaned_reports/llama_finetune_input.json

Finetune Llama-3

In [ ]:
!pip install -q openai==0.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --quiet huggingface_hub
!huggingface-cli login

In [ ]:
!pip install -q bitsandbytes accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.5 MB/s eta 0:00:00


In [ ]:
%%bash

export MODEL_NAME=meta-llama/Meta-Llama-3.1-8B-Instruct
export DATA_PATH=/content/drive/MyDrive/LLM_CXR/cleaned_reports/llama_finetune_input.json
export OUTPUT_DIR=/content/drive/MyDrive/LLM_CXR/LLM_MODEL_FILES/finetuned-pragmatic-llama3

accelerate launch finetune_qLoRA.py \
  --model_name_or_path $MODEL_NAME \
  --data_path        $DATA_PATH \
  --output_dir       $OUTPUT_DIR \
  --per_device_train_batch_size 1 \
  --gradient_accumulation_steps 4 \
  --num_train_epochs 5 \
  --learning_rate 1e-4 \
  --report_to none

Fine-tune with Synthetic Data

In [ ]:
FINETUNE_IMPRESSIONS = "/content/drive/MyDrive/LLM_CXR/cleaned_reports/manually_prepared/FINETUNE_IMPRESSIONS.csv"
FINETUNE_INDICATIONS = "/content/drive/MyDrive/LLM_CXR/cleaned_reports/manually_prepared/FINETUNE_INDICATIONS.csv"

MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"
SYNTH_IMPRESSIONS = "/content/drive/MyDrive/LLM_CXR/synthetic_data/SYNTH_IMPRESSIONS.csv"
SYNTH_INDICATIONS = "/content/drive/MyDrive/LLM_CXR/synthetic_data/SYNTH_INDICATIONS.csv"
SYNTH_DATASET = "/content/drive/MyDrive/LLM_CXR/synthetic_data/SYNTH_DATASET.csv"

In [ ]:
import pandas as pd
imp = pd.read_csv(FINETUNE_IMPRESSIONS)
# Compute P(flag==1) and pairwise corr
freqs = imp.drop(columns=["study_id","report"]).mean()
corrs = imp.drop(columns=["study_id","report"]).corr()

In [ ]:
import numpy as np

n_synth = 1000
p = freqs.values
synth_labels = (np.random.rand(n_synth, len(p)) < p).astype(int)

In [ ]:
!pip install -q transformers accelerate
!huggingface-cli login

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
model     = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
)

In [ ]:
import numpy as np, pandas as pd, uuid

real_impressions = imp.copy()

# sample N fake label vectors (shape: [N, num_conditions])
p = real_impressions.drop(columns=["study_id","report"]).mean().values
N = 1000
synth_labels = (np.random.rand(N, len(p)) < p).astype(int)
cond_names   = real_impressions.columns.drop(["study_id","report"])

rows = []
for vec in synth_labels:
    label_dict = dict(zip(cond_names, vec))
    # synthetic generation prompt:
    findings = ", ".join(k for k,v in label_dict.items() if v) or "no acute findings"
    prompt = (
        f"You are a radiologist. Findings present: {findings}.\n"
        f"Write a concise Impression (1–2 sentences), mentioning positives and stating normal for others.\n\n"
        f"### Impression:\n"
    )
    out = generator(
        prompt,
        max_new_tokens=64,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )[0]["generated_text"]
    # strip off the prompt and any trailing garbage
    imp = out.split("### Impression:")[-1].strip()
    rows.append({"study_id": str(uuid.uuid4()), "report": imp, **label_dict})

synth_imp = pd.DataFrame(rows)
synth_imp.to_csv(SYNTH_IMPRESSIONS, index=False)

In [ ]:
# LLM-paraphrase a seed set:
paraphrase = pipeline("text2text-generation", model=MODEL_ID, device=0)
real_inds = pd.read_csv(FINETUNE_INDICATIONS)
ind_rows = []
for sid, hist in zip(synth_imp.study_id, real_inds.history.sample(len(synth_imp), replace=True)):
    p = paraphrase(f"Rewrite this exam history in a different way:\n\n{hist}",
                   max_new_tokens=64, temperature=0.8)[0]["generated_text"]
    ind_rows.append({"study_id": sid, "history": p})

synth_ind = pd.DataFrame(ind_rows)
synth_ind.to_csv(SYNTH_INDICATIONS, index=False)

In [ ]:
combined = synth_ind.merge(synth_imp, on="study_id")
# drop any empty or obviously malformed rows
combined = combined[combined.report.str.len().between(20,500)]
combined.to_csv(SYNTH_DATASET, index=False)